In [2]:
import pandas as pd
DATA_PATH = '../../data' #Insert path to data here
from sklearn.model_selection import train_test_split

**Si recuperano tutti i valori nel campo ‘city_or_county’ per i quali non è possibile determinare se si riferiscono a una ‘city’ o a una ‘county’.**

In [3]:
cities = pd.read_csv("../../data/uscities.csv")
county = pd.read_csv("../../data/county.csv")
df = pd.read_csv(f"{DATA_PATH}/cluster.csv")
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df.date.dt.year
df = df[(df.year >= 2014) & (df.year <= 2017) ]

county_entries = df[df['city_or_county'].str.contains('county', case=False, na=False)]['city_or_county']
not_recognized = df[(~df.city_or_county.isin(cities.city)) & (~df.city_or_county.isin(county.name)) & (~df.city_or_county.isin(county_entries))].drop_duplicates(subset=['city_or_county'])

**Tramite latitude e longitude recuperiamo la relativa città**

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="prova")
def get_city(row):
    location = geolocator.reverse([row['latitude'], row['longitude']], exactly_one=True).raw['address']
    return location.get('city', None) #Restituisce null se la posizione non si riferisce ad una città

not_recognized['city'] = not_recognized.apply(get_city, axis=1)
not_recognized.to_csv(f"../data/not_recognized.csv", index=False)

**Recuperiamo solo le entry che contengono le città**

In [7]:
additional_cities = pd.read_csv("data/not_recognized.csv")[["city_or_county", "city"]].dropna()
dict_temp = additional_cities.set_index('city_or_county')['city'].to_dict()
dict_additional_cities = additional_cities.set_index('city_or_county')['city'].to_dict()

# Tutte le entry di df['city_or_county'] che sono presenti 
# in additional_cities['city_or_county'] verranno sostituite con il corrispondente valore di additional_cities['city']
df['city_or_county'] = df['city_or_county'].map(dict_additional_cities).fillna(df['city_or_county'])
df.rename(columns={'city_or_county': 'city'}, inplace=True)

In [8]:
df.to_csv(f"data/time_Series_Data.csv", index=False)